In [1]:
import pandas as pd
import numpy as np
import datetime
from haversine import haversine
import os
import json

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2020_Q1/plannder_requests/Margaret_zips_Columbus_OH_20200422'

In [2]:
df_store_list=pd.read_csv("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200401-135137-445.txt",dtype=str,sep="|")
df_store_list=df_store_list[['location_id','address_line_1','address_line_2','city_nm','state_nm','zip_cd','latitude_meas','longitude_meas']]
df_store_list['latitude_meas']=df_store_list['latitude_meas'].astype(float)
df_store_list['longitude_meas']=df_store_list['longitude_meas'].astype(float)
df_store_list['zip_cd']=df_store_list['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
df_store_list=df_store_list[~df_store_list['location_id'].isin(['145','6990'])]


In [3]:
DMA_zip=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dytpe=str,usecols=[0,2],skiprows=1)
DMA_zip=DMA_zip.drop_duplicates()
# DMA_zip.columns=['zip_cd','DMA','ST']
DMA_zip.columns=['zip_cd','DMA']
DMA_zip['zip_cd']=DMA_zip['zip_cd'].astype(str)
DMA_zip['zip_cd']=DMA_zip['zip_cd'].apply(lambda x: x.zfill(5))
DMA_zip=DMA_zip.drop_duplicates()

DMA_zip_ColumbusOH=DMA_zip[DMA_zip['DMA']=="COLUMBUS, OH"].reset_index()
del DMA_zip_ColumbusOH['index']

In [4]:
df_store_list_ColumbusOH=pd.merge(df_store_list,DMA_zip_ColumbusOH,on="zip_cd",how="inner")
df_store_list_ColumbusOH.shape

(21, 9)

In [5]:
TA_zips=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx")
TA_zips=TA_zips.parse("view_by_store",dtype=str)
TA_zips.head(2)

,TA_num,ta_name,location_id,address_line_1,address_line_2,zip_cd,city_nm,state_nm,latitude_meas,longitude_meas,DMA,CTY,all_trans_P_zips,all_trans_S_zips,zips_in_10,trans_P_zips_70_within_TA,trans_S_zips_70_within_TA
0,1,multiple_1,1949,3178 LAVON DR,NaN,75040,GARLAND,TX,32.945015,-96.619194,{'DALLAS-FT. WORTH'},{'DALLAS'},"['75040', '75044', '75048', '75098']","['75089', '75042', '75043', '75041', '75088', ...",[],"['75040', '75098', '75044', '75048']","['75088', '75042', '75082', '75043', '75089', ..."
1,1,multiple_1,1255,1418 W MOORE AVE,NaN,75160,TERRELL,TX,32.738772,-96.299877,{'DALLAS-FT. WORTH'},"{'HUNT', 'KAUFMAN', 'ROCKWALL'}","['75160', '75126', '75161']","['75142', '75169', '75474', '75143', '75114', ...",[],"['75126', '75161', '75160']","['75142', '75169', '75150', '75117', '75143', ..."


In [6]:
df_temporary=TA_zips[['location_id','trans_P_zips_70_within_TA','trans_S_zips_70_within_TA','zips_in_10']]
df_zip_by_store=pd.DataFrame()

for ind,row in df_temporary.iterrows():
    location_id=row['location_id']
    P_zips=eval(row['trans_P_zips_70_within_TA'])
    S_zips=eval(row['trans_S_zips_70_within_TA'])
    zip_10=eval(row['zips_in_10'])
    

    df_P=pd.DataFrame(zip([location_id]*len(P_zips),P_zips))
    if len(df_P)>0:
        df_P.columns=['location_id','zip_cd']
        df_P['zip_type']="P"
        
    df_S=pd.DataFrame(zip([location_id]*len(S_zips),S_zips))
    if len(df_S)>0:
        df_S.columns=['location_id','zip_cd']
        df_S['zip_type']="S"
    
    df_10=pd.DataFrame(zip([location_id]*len(zip_10),zip_10))
    if len(df_10)>0:
        df_10.columns=['location_id','zip_cd']
        df_10['zip_type']="zip_10"
    
    df_zip_by_store=df_zip_by_store.append(df_P).append(df_S).append(df_10)

In [7]:
df_store_list_ColumbusOH=df_store_list_ColumbusOH.rename(columns={"zip_cd":"store_zip"})
df_store_ColumbusOH_zip=pd.merge(df_store_list_ColumbusOH,df_zip_by_store,on="location_id",how="left")
df_store_ColumbusOH_zip.head(2)

df_store_ColumbusOH_zip_existing=df_store_ColumbusOH_zip[pd.notnull(df_store_ColumbusOH_zip['zip_cd'])]

In [8]:
df_new_store=df_store_ColumbusOH_zip[pd.isnull(df_store_ColumbusOH_zip['zip_cd'])]
del df_new_store['zip_cd']
del df_new_store['zip_type']
df_new_store=df_new_store.reset_index()
del df_new_store['index']

df_all_new_zip=pd.DataFrame()
for i,row in df_new_store.iterrows():
    store_coor=(row['latitude_meas'],row['longitude_meas'])
    store_num=row['location_id']
    list_store_zip=[]
    for zip_cd, v in zip_centers.items():
        dist=haversine(store_coor,v,unit="mi")
        if dist<=10:
            list_store_zip.append(zip_cd)
    df=pd.DataFrame({"zip_cd":list_store_zip,"zip_type":["zip_10"]*len(list_store_zip)},index=[store_num]*len(list_store_zip))
    df=df.reset_index().rename(columns={"index":"location_id"})
    df_all_new_zip=df_all_new_zip.append(df)
    
df_new_store=pd.merge(df_new_store,df_all_new_zip,on="location_id",how="left")
df_new_store.shape

(11, 11)

In [9]:
df_all_store_ColumbusOH_detail=df_store_ColumbusOH_zip_existing.append(df_new_store)

In [10]:
df_unqiue_zip=df_all_store_ColumbusOH_detail[['zip_cd']].drop_duplicates()

In [11]:
writer=pd.ExcelWriter("./BL_zip_for_all_ColumbusOH_DMA_stores_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_unqiue_zip.to_excel(writer,"unique_zip",index=False)
df_all_store_ColumbusOH_detail.to_excel(writer,"details",index=False)
writer.save()